In [204]:
from lxml import etree
import os
def extract(path, pdf_content_list,bucket_links):
    xml_file_path = os.path.abspath(path)

    if os.path.exists(xml_file_path):
        tree = etree.parse(xml_file_path)
        root = tree.getroot()
        namespaces = {
            'tei': 'http://www.tei-c.org/ns/1.0',
            'xlink': 'http://www.w3.org/1999/xlink'
        }

        def get_first_item(xpath_result):
            if xpath_result:
                xpath_result[0] = xpath_result[0].replace('\n', '').replace('\t','')
                return f"{xpath_result[0]}"  
            else:
                return "No Data"

        # Extract metadata using XPath expressions
        metadata_dict = {
            "Title": get_first_item(root.xpath('//tei:titleStmt/tei:title[@level="a" and @type="main"]/text()', namespaces=namespaces)),
            "Publisher": get_first_item(root.xpath('//tei:publicationStmt/tei:publisher/text()', namespaces=namespaces)),
            "AvailabilityStatus": get_first_item(root.xpath('//tei:availability/@status', namespaces=namespaces)),
            "Analytic": get_first_item(root.xpath('//tei:analytic/text()', namespaces=namespaces)),
            "ImprintedDate": get_first_item(root.xpath('//tei:imprint/tei:date/text()', namespaces=namespaces)),
            "AppInfoDescription": get_first_item(root.xpath('//tei:application/tei:desc/text()', namespaces=namespaces)),
            "Abstract": get_first_item(root.xpath('//tei:profileDesc/tei:abstract/tei:p/text()', namespaces=namespaces)),
        }
        metadata_dict['s3_bucket_link'] = bucket_links[path]

        pdf_content_list.append(metadata_dict)
    else:
        print(f"The file {xml_file_path} does not exist.")



In [205]:
paths = ['./resources/metadata/Level1_combined.grobid.tei.xml',
         './resources/metadata/Level2_combined.grobid.tei.xml',
         './resources/metadata/Level3_combined.grobid.tei.xml',
        ]
bucket_links = {
    "./resources/metadata/Level1_combined.grobid.tei.xml" : "s3://cfainstitute-learning-outcomes-raw/grobid/Grobid_RR_2024_Level1_combined.txt",
    "./resources/metadata/Level2_combined.grobid.tei.xml": "s3://cfainstitute-learning-outcomes-raw/grobid/Grobid_RR_2024_Level2_combined.txt",
    "./resources/metadata/Level3_combined.grobid.tei.xml": "s3://cfainstitute-learning-outcomes-raw/grobid/Grobid_RR_2024_Level3_combined.txt"
}
pdf_content_list=[]
for path in paths:
        extract(path,pdf_content_list,bucket_links)

In [206]:
import pandas as pd

md = pd.DataFrame(pdf_content_list)

In [207]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()

True

In [208]:
snowflake_account = os.getenv('snowflake_account')
snowflake_user = os.getenv('snowflake_user')
snowflake_password = os.getenv('snowflake_password')
snowflake_warehouse = os.getenv('snowflake_warehouse')
snowflake_schema = os.getenv('snowflake_schema')
snowflake_database = 'your_database'

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=snowflake_user,
        password=snowflake_password,
        account_identifier=snowflake_account,
    )
)
connection = engine.connect()


In [209]:
target_table = 'metadata_grobid'

target_database = 'MetadataDB'


create_database_query = f"CREATE DATABASE IF NOT EXISTS {target_database}"

connection.execute(create_database_query)
use_database_query = f"USE DATABASE {target_database}"
connection.execute(use_database_query)
connection.execute("USE WAREHOUSE TEST")


In [210]:
headers = md.columns.tolist()
columns_definition = ', '.join([f"{header} STRING" for header in headers])
connection.execute(f"""
    CREATE TABLE IF NOT EXISTS metadata_grobid (
        {columns_definition}
    )
    """)

In [211]:
md.to_csv('./resources/metadata/metadata_grobid.csv', index=False)

In [212]:
connection.execute(f"TRUNCATE TABLE metadata_grobid")

connection.execute(f"PUT file://resources/metadata/metadata_grobid.csv @%metadata_grobid")


In [213]:
connection.execute(f"COPY INTO metadata_grobid ON_ERROR=CONTINUE FILE_FORMAT = (FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER=1 PARSE_HEADER = FALSE)")